# Get Control Subreddits

In [1]:
# -*- coding: utf-8 -*-

In [2]:
import re
import os
import sys
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import json
from scipy import stats
import csv
import pprint
from collections import defaultdict
import time
import datetime
import math
import glob
import praw
# import matplotlib.pyplot as plt; plt.rcdefaults()
# from IPython.display import Markdown, display

In [3]:
pd.options.display.max_rows = 1999

In [4]:
TRP_USERS = "/mnt/storage/quarantine/data/candidate_control_subs_TRP_users.csv"
TOTAL_USERS = "/mnt/storage/quarantine/data/candidate_control_subs_unique_users.csv"
QUARANTINED_SUBS_LIST = "/mnt/storage/quarantine/data/list-quarantined-subs.txt"

In [5]:
# reddit = praw.Reddit(client_id='ILLOrLiBZiiZ7w', \
#                     client_secret='D3M0rtwwmLf0QtOZszZ-4aZL5g8', \
#                     user_agent='sugarfennel', \
#                     username='username', \
#                     password='password')

In [6]:
df = pd.read_csv(TRP_USERS)
TOTAL_df = pd.read_csv(TOTAL_USERS)

In [7]:
df = df.join(TOTAL_df.set_index('Subreddit'), on='Subreddit')

In [8]:
df['TRP_users/Total_users'] = df['TRP_users']/df['Total_users']

In [9]:
#filter out subreddits that are present in the list of all quarantined subreddits compiled from r/reclassified
quarantined_subs_list = pd.read_csv(QUARANTINED_SUBS_LIST, names = ['subreddit'])['subreddit'].tolist()
quarantined_subs_list.append('debatefascism')
df = df[~df.Subreddit.str.lower().isin(quarantined_subs_list)]

In [10]:
g = df[df.Total_users >= 100].sort_values(by=['TRP_users/Total_users'], ascending=False)[:100]
topSubs = g['Subreddit'].tolist()

### Using Reddit API to check the 'active' status of each control subreddit

In [11]:
# for sub in topSubs:
#     try:
#         subSubscribers = reddit.subreddit(sub).subscribers
#     except Exception as e:
#         print (sub)

# List of top 100 candidates for control subreddits based on the proportion of users who come from r/theredpill. We do not include the quarantined subreddits in this list.

In [12]:
g

,Subreddit,TRP_users,Total_users,TRP_users/Total_users
1,asktrp,2524,11434,0.220745
72,askMRP,183,1095,0.167123
847,The30DayChallenge,19,149,0.127517
86,marriedredpill,152,1229,0.123678
406,RPChristians,38,310,0.122581
27,PurplePillDebate,343,3524,0.097333
39,WhereAreAllTheGoodMen,272,3453,0.078772
78,RedPillWomen,169,2417,0.069921
343,DarkEnlightenment,44,638,0.068966
606,BattleOfTheSexes,26,413,0.062954


In [13]:
g.to_csv("control_subreddits.csv")

In [14]:
g['Subreddit'].to_csv("/mnt/storage/quarantine/data/top-100-controls-subreddits.csv", index = False, header = False)

### Printing out top subreddits for Google BigQuery

In [15]:
quot = "\""
text = ""
for sub in topSubs:
    text = text + quot + sub + quot + ","

lb = "("
rb = ")"
print(lb + text[:-1] + rb)

("asktrp","askMRP","The30DayChallenge","marriedredpill","RPChristians","PurplePillDebate","WhereAreAllTheGoodMen","RedPillWomen","DarkEnlightenment","BattleOfTheSexes","MGTOW2","BlackPillScience","Semenretention","MGTOW","askseddit","seduction","steroids","Testosterone","AJelqForYou","MensRights","coldshowers","TheBluePill","PEDs","AsianMasculinity","nattyorjuice","truerateme","leangains","Anarcho_Capitalism","sociopath","Trufemcels","PussyPass","naturalbodybuilding","short","Incelselfies","Stronglifts5x5","C_S_T","SocialJusticeInAction","JordanPeterson","phenibut","NoFap","Shitstatistssay","Brogress","hapas","metacanada","pussypassdenied","ShitPoliticsSays","datingoverthirty","IncelsWithoutHate","GoldandBlack","KotakuInAction","Stoicism","zen","Nootropics","StackAdvice","StopGaming","nSuns","pornfree","tressless","AskThe_Donald","Supplements","EqualAttraction","TheNewRight","BPDlovedones","entp","CapitalismVSocialism","afinil","martialarts","antinatalism","ForeverAlone","bigdickproble